# 実習06 機械学習モデル作成（前半のまとめ）
---
前半のまとめとして、ここまで学んだことを可能な限り使用して、機械学習のモデルを作成する。

大まかな流れは以下である。

1.   問題の設定、学習分類を決める
2.   データの準備
4.   前処理の検討と作成
4.   機械学習モデルの選択と設計
5.   学習、テスト、判断


In [155]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1. 問題の設定

### 背景の設定
日本と北米で販売されているビデオゲームをEU向けに展開するかどうかの判断をすべく、ゲームのハードやジャンル、日本と北米での売り上げなどの情報から、EUでの売り上げを予測したい。

### 具体的なデータと学習分類
今回は、2016年12月時点でのビデオゲーム総売り上げのデータセット（VideoGamesSales_ansi.csv）を使用し、様々な情報からEUでの売り上げを本数を予測するAIモデルを作成する。

**[課題]**学習分類（学習の種類）は何ですか？<br>
**[　教師あり学習、分類　]**



## 2. データセットの準備
### 2-1. CSVファイルの読み込み

CSVファイルをGoogle Colab上にアップロード、または作業ディレクトリに移動またはコピーし、pandasのDataFrameとして読み込む。
読み込む前に直接中身を確認してもよい。

**※今回はindex_colは指定しない。**<br>
**※CSVファイルがsjisのため、encodingを指定する**
```
# CSVファイルの読み込み
import pandas as pd
df = pd.read_csv("VideoGamesSales_ansi.csv", encoding='shift_jis')
```

In [156]:
# CSVファイルの読み込み
import pandas as pd
df = pd.read_csv("VideoGamesSales_ansi.csv", encoding='shift_jis')

In [157]:
#表示して確認してもよい
df

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16714,Samurai Warriors: Sanada Maru,PS3,2016.0,Action,Tecmo Koei,0.00,0.00,0.01,0.00,NaN,NaN,NaN,NaN,NaN,NaN
16715,LMA Manager 2007,X360,2006.0,Sports,Codemasters,0.00,0.01,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN
16716,Haitaka no Psychedelica,PSV,2016.0,Adventure,Idea Factory,0.00,0.00,0.01,0.00,NaN,NaN,NaN,NaN,NaN,NaN
16717,Spirits & Spells,GBA,2003.0,Platform,Wanadoo,0.01,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN


### 2-2. データの性質を理解する
各データの性質（数値なのかカテゴリ変数なのか文字列なのか、など）を理解し、目的変数（予測すべきデータ）がどれかを確認する。

> Name：ゲームのタイトル**[文字列]**<br>
> Platform：ハード**[カテゴリ変数]**<br>
> Year_of_Release：リリース年**[数値またはカテゴリ変数]**<br>
> Genre：ジャンル**[カテゴリ変数]**<br>
> Publisher：発売元**[カテゴリ変数]**<br>
>	NA_Sales：北米での売り上げ（単位：百万本）**[数値]**<br>
> EU_Sales：EUでの売り上げ（単位：百万本）**[数値]**<br>
>	JP_Sales：日本での売り上げ（単位：百万本）**[数値]**<br>
> Critic_Score：レビューサイトスタッフによる平均スコア**[数値]**<br>
> Critic_Count：Critic_Scoreの数**[数値]**<br>
> User_Score：レビューサイトユーザによる平均スコア**[数値]**<br>
> User_Count：User_Scoreの数**[数値]**<br>
> Developer：開発元**[カテゴリ変数]**<br>
> Rating：ESRBによる年齢区分**[カテゴリ変数]**<br>

## 2-3. 欠損値の確認と方針の検討
まずは欠損値があるかを確認する。（個数を数える）

In [158]:
#欠損値の数を確認
df.isnull().sum()

,0
Name,2
Platform,0
Year_of_Release,269
Genre,2
Publisher,54
NA_Sales,0
EU_Sales,0
JP_Sales,0
Other_Sales,0
Critic_Score,8582


出力以外の各列について、処理をどうするか決める。

**明確な正解はありません。可能な選択肢の中でいろいろ試してみてください。**<br>
**モデルの学習にかなり時間がかかる可能性あるので、最初は予測に使用する列（特にダミー変数化して使用する列）を少なくすると良いかもしれません**

**【課題】各列の処理をどうするかを決めて記述する。**<br>
* 複数列に同じ前処理を行う場合はまとめて記述してよい。
* 処理しない列はすべて削除する場合などは、列名に「その他の列」と書いてまとめてよい。

### □列名
削除：Name、Critic_Score、Critic_Count、Developer、Rating<br>

### □列名、列名、・・・
平均値で補完 => ダミー変数にする:<br>
Platform, Genre, Publisher<br>

平均値で補完 => ### 正規化・標準化:<br>
User_Score, User_Count, Year_of_Release, NA_Sales, JP_Sales, Other_Sales<br>
### □その他の列
処理を書く
（上記に当てはまらないものすべてに同じ処理をする場合）

###2-4. 入力と出力、訓練用とテスト用に分ける
まずは、「欠損値がある『行』を学習に含めない」に該当するものを削除する。（必ずしも実施する必要はない。）
```
変数名 = df.dropna(subset=['列名'])
```
**※実施する場合は、入力と出力に分ける前に実施する**

In [159]:
# 欠損値がある「行」を削除（処理を実施する場合）
df.dropna(subset=['User_Score'], inplace=True)
df.dropna(subset=['User_Count'], inplace=True)

入力と出力、訓練用とテスト用に分割する。<br>
**train_test_split関数のrandom_stateは学籍番号にしてください。**

In [187]:
# 出力の列を抽出
x = df.drop('EU_Sales',axis=1)
# 入力
y = df['EU_Sales']

# 訓練用とテスト用に分ける
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.7, random_state=2220042)

## 3. 前処理の作成
「前処理＞機械学習モデル」のパイプラインを作成する準備として、必要な前処理を準備する。


In [182]:
# 必要な物を先にインポートしておいてもよい
from sklearn.impute import SimpleImputer  #欠損値の補完
from sklearn.preprocessing import OrdinalEncoder #数値ラベルに変換
from sklearn.preprocessing import OneHotEncoder #ダミー変数に変換
from sklearn.preprocessing import MinMaxScaler #正規化
from sklearn.compose import ColumnTransformer #列ごとに違う処理をする
from sklearn.pipeline import Pipeline # pipelineの作成
from sklearn.svm import SVR

前処理をColumnTransformerで作成する。1列に複数の処理を行う場合は、ColumnTransformerの中でPipelineを使用する。

In [225]:
preprocessor = ColumnTransformer(
    transformers=[
        # 数値データの処理（欠損値を平均値で補完）
        ('numS2', Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', MinMaxScaler())
        ]), ['User_Score', 'User_Count', 'Year_of_Release', 'NA_Sales', 'JP_Sales', 'Other_Sales']),

        ('categorical', Pipeline([
            ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
            ('onehot', OneHotEncoder(handle_unknown='ignore'))
        ]), ['Platform', 'Genre', 'Publisher']),

        ('drop_col', 'drop', ['Name','Critic_Score', 'Critic_Count', 'Rating','Developer'])
    ],
    remainder='drop'
)

fit_transformにx_train、x_testを入力して、エラーが出ないことを確認する。<br>
* <～x～ sparse matrix of ～>が表示される場合はエラーではない。
* エラーが出る場合は、元データを編集するか、Noneなどに置き換える。<br>
（score列に数値以外の「tbd」が入っている箇所がある）
```
# 置き換える場合の例
x_train = x_train.replace("tbd", None)
x_test = x_test.replace("tbd", None)
```

試しに、適当なアルゴリズムでモデル作成し学習してみてもよい。

In [191]:
full_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', SVR())
])

# モデルの学習
full_pipeline.fit(x_train, y_train)

# モデルの評価
accuracy = full_pipeline.score(x_test, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.3323688347313539


## 4. 機械学習モデルの作成と設計
### 4-1. アルゴリズムの選択
使用可能な全てのアルゴリズムを試し、精度の高そうなアルゴリズムを選択する。（実習04を参照）<br>
**特に、入力の次元が大きくなるとかなり時間がかかる。また、うまく入力ができない場合がある。**

*   学習分類に対応した全てのアルゴリズムを取得
*   各アルゴリズムに対して精度を比較<br>
    **ループ内でモデルを作成する場合も、前処理→モデルのPipelineを作成すること。**
*   精度の高そうなアルゴリズムを選択

（難しければ）
[チートシート](https://scikit-learn.org/stable/tutorial/machine_learning_map/)
などからアルゴリズムを選択する。余裕があればグリッドサーチでパラメータを選択する。<br>
**アルゴリズムを選択したら5へ**



In [217]:
#使用可能なアルゴリズムの取得
from sklearn.utils import all_estimators
estimators = all_estimators(type_filter='regressor')

In [221]:
# forループを使用して各アルゴリズムのscoreを表示する
for name, ClassifierClass in estimators:
  if name == "QuantileRegressor" or name == "GaussianProcessRegressor":
    continue
  try:
    model = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', ClassifierClass())
    ])

    model.fit(x_train, y_train)
    score = model.score(x_test, y_test)
    if score > 0.7:
      print(name,"のscore＝",score)

  # エラーが出ても止まらないようにする
  except Exception as e:
    pass


ElasticNetCV のscore＝ 0.7137232516458425
ExtraTreesRegressor のscore＝ 0.7518782125422905
GradientBoostingRegressor のscore＝ 0.7080299652665327


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_huber.py:342: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


HuberRegressor のscore＝ 0.7487066407946175
KernelRidge のscore＝ 0.7260718578732299
LassoCV のscore＝ 0.7215223651648649
LinearRegression のscore＝ 0.7411236203992506


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVR のscore＝ 0.7318800691454441


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_regression.py:509: UserWarning: One or more samples have no neighbors within specified radius; predicting NaN.
  warnings.warn(empty_warning_msg)


Ridge のscore＝ 0.7259502008477117
RidgeCV のscore＝ 0.7259486842227572
TransformedTargetRegressor のscore＝ 0.7411236203992506


**[課題] 選択したアルゴリズムは：[　regressor　]** <br>

## 5. 学習、テスト、判断
### 5-1. 学習
改めてモデルをPipelineで作成し、訓練用データを使用して、fit()でモデルを学習させる。

In [222]:
# 改めてモデル作成
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline

full_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', Ridge())
])


In [223]:
# fitで学習
full_pipeline.fit(x_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numS2',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['User_Score', 'User_Count',
                                                   'Year_of_Release',
                                                   'NA_Sales', 'JP_Sales',
                                                   'Other_Sales']),
                                                 ('categorical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='unknown',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Platform', 'Genre',
                                                   'Publisher']),
                                                 ('drop_col', 'drop',
                                                  ['Name', 'Critic_Score',
                                                   'Critic_Count',
                                                   'Rating'])])),
                ('regressor', Ridge())])

### 5-2. テスト、判断
テストデータとscoreを使用して、このモデルでよいか判断する。

In [224]:
# scoreの計算
full_pipeline.score(x_test, y_test)

0.7259502008477117

## 提出物など

実習が終わったら、以下を教員にチェックしてもらうこと。

*   2-3の最後　各列の処理
*   4-1の最後　 選択したアルゴリズム
*   5-2の最後　テストデータのscore

その後、このファイルをipynbでダウンロードして提出してください。（ファイル名のクラス番号氏名を確認すること。）

